# Compustat World Firms Industry Classification

This file was created to make parallesim easier.

##  Import Libraries and Load Datasets

In [18]:
# Data analysis
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.stats import uniform, loguniform
from skopt.space import Real, Categorical, Integer
import pickle


# Preprocessing & Splitting
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GroupShuffleSplit, StratifiedGroupKFold
from sklearn.decomposition import PCA

# Modeling
from skopt import BayesSearchCV 
import xgboost
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Utils
from utils import *


In [19]:
compustat = pd.read_pickle('../data/compustat_ftreng.pkl')

In [20]:
compustat.shape

(2748178, 139)

## Model Fitting

### Preprocessing

In [21]:
year_ftrs = ['fyearq','fqtr']
num_ftrs = ['saleq', 'gpq', 'oiadpq', 'oibdpq', 'cogsq',
       'xoprq', 'atq', 'seqq', 'dlcq', 'dlttq', 'capxy', 'oancfy', 'gpm',
       'opm', 'ocfm', 'roa', 'roe', 'cd_ratio', 'ca_ratio', 'fca_ratio',
       'fce_ratio', 'fcd_ratio', 'fcs_ratio', 'tat', 'cr', 'tdr', 'der',
       'gpm_lag1', 'gpm_lag2', 'gpm_lag3', 'gpm_lag4', 'opm_lag1',
       'opm_lag2', 'opm_lag3', 'opm_lag4', 'ocfm_lag1', 'ocfm_lag2',
       'ocfm_lag3', 'ocfm_lag4', 'roa_lag1', 'roa_lag2', 'roa_lag3',
       'roa_lag4', 'roe_lag1', 'roe_lag2', 'roe_lag3', 'roe_lag4',
       'fca_ratio_lag1', 'fca_ratio_lag2', 'fca_ratio_lag3',
       'fca_ratio_lag4', 'fce_ratio_lag1', 'fce_ratio_lag2',
       'fce_ratio_lag3', 'fce_ratio_lag4', 'fcd_ratio_lag1',
       'fcd_ratio_lag2', 'fcd_ratio_lag3', 'fcd_ratio_lag4',
       'fcs_ratio_lag1', 'fcs_ratio_lag2', 'fcs_ratio_lag3',
       'fcs_ratio_lag4', 'tat_lag1', 'tat_lag2', 'tat_lag3', 'tat_lag4',
       'cr_lag1', 'cr_lag2', 'cr_lag3', 'cr_lag4', 'tdr_lag1', 'tdr_lag2',
       'tdr_lag3', 'tdr_lag4', 'der_lag1', 'der_lag2', 'der_lag3',
       'der_lag4', 'gpm_mean_4Q', 'gpm_std_4Q', 'gpm_mean_8Q',
       'gpm_std_8Q', 'opm_mean_4Q', 'opm_std_4Q', 'opm_mean_8Q',
       'opm_std_8Q', 'ocfm_mean_4Q', 'ocfm_std_4Q', 'ocfm_mean_8Q',
       'ocfm_std_8Q', 'roa_mean_4Q', 'roa_std_4Q', 'roa_mean_8Q',
       'roa_std_8Q', 'roe_mean_4Q', 'roe_std_4Q', 'roe_mean_8Q',
       'roe_std_8Q', 'fca_ratio_mean_4Q', 'fca_ratio_std_4Q',
       'fca_ratio_mean_8Q', 'fca_ratio_std_8Q', 'fce_ratio_mean_4Q',
       'fce_ratio_std_4Q', 'fce_ratio_mean_8Q', 'fce_ratio_std_8Q',
       'fcd_ratio_mean_4Q', 'fcd_ratio_std_4Q', 'fcd_ratio_mean_8Q',
       'fcd_ratio_std_8Q', 'fcs_ratio_mean_4Q', 'fcs_ratio_std_4Q',
       'fcs_ratio_mean_8Q', 'fcs_ratio_std_8Q', 'tat_mean_4Q',
       'tat_std_4Q', 'tat_mean_8Q', 'tat_std_8Q', 'cr_mean_4Q',
       'cr_std_4Q', 'cr_mean_8Q', 'cr_std_8Q', 'tdr_mean_4Q',
       'tdr_std_4Q', 'tdr_mean_8Q', 'tdr_std_8Q', 'der_mean_4Q',
       'der_std_4Q', 'der_mean_8Q', 'der_std_8Q']

#cat_ftrs = ['loc','curcdq']  # maybe not include country for now
cat_ftrs = ['curcdq'] 


In [22]:
minmax_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())])
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=True, handle_unknown='ignore'))])
preprocessor1 = ColumnTransformer(
    transformers=[
        ('minmax', minmax_transformer, year_ftrs),
        ('num', numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs)])

numeric_transformer2 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))])
categorical_transformer2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))])
preprocessor2 = ColumnTransformer(
    transformers=[
        ('minmax', minmax_transformer, year_ftrs),
        ('num', numeric_transformer2, num_ftrs),
        ('cat', categorical_transformer2, cat_ftrs)])

Preprocessor suitable for XGBoost, will produce missing values

Preprocessor2 is for random forest, LR, etc. It will fill missing values with 0

### Hyperparameter tuning w/ RandomizedSearchCV

In [29]:

def ML_BayesSearch_CV(X_og, y_og, groups_og, preprocessor, ML_algo, search_space, sample_size=None):
    # Loop for 5 random states
    # in each loop, split, preprocess, fit, and score
    random_states = [377, 575, 610, 777, 233]

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    reports = []
    cms = []
    best_param = []
    best_score = []

    for i in range(1):
        this_rs = random_states[i]
        
        # Split into [train,val] and test sets (80%, 20%)
        sgss = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=this_rs)
        splits = enumerate(sgss.split(X_og, y_og, groups_og))

        print('Splitting dataset into train-val and test sets (80%, 20%)')
        i, (train_val_idx, test_idx) = next(splits)
        X_train_val, X_test = X_og.iloc[train_val_idx], X_og.iloc[test_idx]
        y_train_val, y_test = y_og.iloc[train_val_idx], y_og.iloc[test_idx]
        group_train_val, group_test = groups_og.iloc[train_val_idx], groups_og.iloc[test_idx]


        #full_length = len(y_train_val)
        # After getting test set, downsample train-val 
        if sample_size is None:
            pass
        elif sample_size <= 0.3*0.8:
            print('Sampling dataset for hyperparameter tuning, sample size of train set (80%): ', sample_size*100,'%')
            flat_idx = downsample_classes(y_train_val, group_train_val, random_state=this_rs)
            flat_X, flat_y, flat_groups = X_train_val.iloc[flat_idx], y_train_val.iloc[flat_idx], group_train_val.iloc[flat_idx]
            flat_prop = len(flat_idx)/len(y_train_val)
            if int(flat_prop/sample_size)<=1: # no need to do more sampling
                X, y = flat_X, flat_y
                groups = flat_groups
            else:
                sampler = StratifiedGroupKFold(n_splits=int(flat_prop/sample_size), shuffle=True, random_state=this_rs)
                splits = enumerate(sampler.split(flat_X, flat_y, flat_groups))
                i, (_, sampled_idx) = next(splits)
                X, y = flat_X.iloc[sampled_idx], flat_y.iloc[sampled_idx]
                groups = flat_groups.iloc[sampled_idx]
        else:
            pass
        X_train_val, y_train_val, group_train_val = X, y, groups
                

        # Split [train,val] into train and val sets (64%, 16%)
        sgss_cv = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=this_rs)
        pipe = make_pipeline(preprocessor, 
                             PCA(n_components=0.95, random_state=this_rs),
                             ML_algo)
        print('Random State Loop: ', i+1)
        print('Random State: ', this_rs)
        print('***Start Bayes Search***')

        bayes_search = BayesSearchCV(estimator=pipe,
                                    search_spaces=search_space,
                                    n_iter=20,  # Number of iterations
                                    cv=sgss_cv,       # Cross-validation strategy
                                    n_jobs=-1,  # Use all available cores
                                    scoring='f1_weighted',
                                    verbose=1,
                                    random_state=this_rs)


        bayes_search.fit(X_train_val, y_train_val, groups=group_train_val)        
    
        # Make predictions and calculate accuracy on the test set
        y_pred = bayes_search.predict(X_test)
        
        accuracy = accuracy_score(*vote_pred(y_test, y_pred, group_test))
        precision = precision_score(*vote_pred(y_test, y_pred, group_test), average="weighted")
        recall = recall_score(*vote_pred(y_test, y_pred, group_test), average="weighted")
        f1 = f1_score(*vote_pred(y_test, y_pred, group_test), average="weighted")

        report = classification_report(*vote_pred(y_test, y_pred, group_test))
        cm = confusion_matrix(*vote_pred(y_test, y_pred, group_test))

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        reports.append(report)
        cms.append(cm)
        best_param.append(bayes_search.best_params_)
        best_score.append(bayes_search.best_score_)
        
    return accuracy_scores, precision_scores, recall_scores, f1_scores, cms, best_param, best_score

In [30]:
compustat[cat_ftrs] = compustat[cat_ftrs].astype(str)

groups = compustat['gvkey']
y = compustat['gsector_num']
X = compustat.drop(['gvkey','gsector','gsector_num','datafqtr'], axis=1)

X.shape

(2748178, 135)

#### XGBoost

#### Random Forest

#### Logistic Regression (Multi-class)

In [31]:
search_space_lr = {
    'logisticregression__C': Real(1e-4, 1e+4, 'log-uniform'),
    'logisticregression__l1_ratio': Real(0, 1, 'uniform'),
    'logisticregression__solver': Categorical(['saga']),
    'logisticregression__max_iter': Integer(5000, 10000)  
}

lr_clf = LogisticRegression(multi_class='multinomial', penalty='elasticnet')
acc_lr, pre_lr, rec_lr, f1_lr, \
    cm_lr, params_lr, bs_lr = ML_BayesSearch_CV(X, y, groups, preprocessor2, 
                                                lr_clf, search_space_lr, sample_size=0.01)
    
lr_results = {
    "accuracy": acc_lr,
    "precision": pre_lr,
    "recall": rec_lr,
    "f1_score": f1_lr,
    "conf_matrix": cm_lr,
    "params": params_lr,
    "best_score": bs_lr
}
with open('../results/lr_results.pkl', 'wb') as file:
    pickle.dump(lr_results, file)

Splitting dataset into train-val and test sets (80%, 20%)
Sampling dataset for hyperparameter tuning, sample size of train set (80%):  1.0 %
Random State Loop:  1
Random State:  377
***Start Bayes Search***
Fitting 4 folds for each of 1 candidates, totalling 4 fits


/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 4 folds for each of 1 candidates, totalling 4 fits


/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits


...

/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits


/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 4 folds for each of 1 candidates, totalling 4 fits


/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/miniconda3/envs/data1030/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [32]:
lr_results

{'accuracy': [0.23084754390430134],
 'precision': [0.2819775283008166],
 'recall': [0.23084754390430134],
 'f1_score': [0.2119857138324001],
 'conf_matrix': [array([[174,  55,   5, 155,   1,  37,  20,  26,  43, 126,  33],
         [ 83, 260,  41, 496,   8,  94,  30,  81, 107, 240,  58],
         [ 81,  64,  88, 942,  28,  34, 125, 182, 183, 322,  99],
         [ 61,  66,  58, 777,  24,  16,  75, 233, 204, 174,  99],
         [ 27,  23,  26, 375,   9,  12,  31,  95,  91,  85,  31],
         [ 40, 131,  13, 226,   2, 143,  39, 286, 216,  63,  95],
         [ 11,  13,   7,  96,   5,   5, 390,  29,  47,  26, 116],
         [ 53,  75,  34, 606,  13,  52, 111, 524, 245, 121,  43],
         [ 35,  21,   8, 126,   7,  24,  33,  86, 110,  67,  28],
         [ 23,   3,   5,  43,   0,   1,  14,  10,  12, 159,  27],
         [ 10,   1,   3,  21,   0,   4,   9,   1,   3,  17,  87]])],
 'params': [OrderedDict([('logisticregression__C', 0.014502448000246788),
               ('logisticregression__l1_r

In [33]:
lr_results = pickle.load(open('../results/lr_results.pkl', 'rb'))
lr_results

{'accuracy': [0.3076923076923077, 0.0],
 'precision': [0.3541669868592946, 0.0],
 'recall': [0.3076923076923077, 0.0],
 'f1_score': [0.3007782873226413, 0.0],
 'conf_matrix': [array([[6, 1, 1, 0, 2, 1, 0, 0, 2, 4, 1],
         [3, 2, 1, 0, 0, 2, 1, 1, 2, 0, 1],
         [0, 4, 1, 0, 2, 0, 0, 2, 0, 2, 1],
         [0, 2, 0, 1, 0, 0, 0, 2, 2, 0, 1],
         [0, 3, 2, 0, 1, 0, 0, 1, 3, 3, 3],
         [0, 0, 0, 0, 0, 4, 0, 1, 2, 2, 0],
         [0, 1, 1, 0, 0, 2, 8, 0, 0, 0, 0],
         [1, 3, 2, 0, 1, 6, 1, 6, 2, 0, 1],
         [1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1],
         [1, 3, 0, 0, 1, 0, 1, 0, 0, 8, 1],
         [1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 6]]),
  array([[0, 0],
         [1, 0]])],
 'params': [OrderedDict([('logisticregression__C', 0.038024176463958406),
               ('logisticregression__l1_ratio', 0.0),
               ('logisticregression__max_iter', 10000),
               ('logisticregression__solver', 'saga')]),
  OrderedDict([('logisticregression__C', 374.41497379557813),
